In [1]:
import os

In [2]:
os.chdir("/Users/SaiTeja/PROJECTS/MLCICD")

In [3]:
%pwd

'c:\\Users\\SaiTeja\\PROJECTS\\MLCICD'

In [24]:
%pwd

'c:\\Users\\SaiTeja\\PROJECTS\\MLCICD'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        # 🔹 Debug: Print loaded YAML
        print("DEBUG: Loaded Config =", self.config)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # 🔹 Debug: Print values retrieved from YAML
        print(f"DEBUG: source_URL = {config.get('source_URL')}")
        print(f"DEBUG: local_data_file = {config.get('local_data_file')}")
        print(f"DEBUG: unzip_dir = {config.get('unzip_dir')}")

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [42]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()

[2025-03-02 19:08:24,275: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-02 19:08:24,277: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-02 19:08:24,280: INFO: common: created directory at: artifacts]
DEBUG: Loaded Config = {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/SAITEJAgitrez2/datasets/raw/refs/heads/main/samsumdata.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformat

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
import shutil
import subprocess

In [34]:
import urllib.request

url = "https://github.com/SAITEJAgitrez2/datasets/raw/refs/heads/main/samsumdata.zip"
local_file = "artifacts/data_ingestion/data.zip"

urllib.request.urlretrieve(url, local_file)
print("✅ File downloaded successfully")


✅ File downloaded successfully


In [36]:
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        """Downloads a file from the given URL if it does not already exist."""
        local_file = Path(self.config.local_data_file)

        if local_file.exists() and local_file.stat().st_size > 0:
            logger.info(f"✅ File already exists: {local_file} ({get_size(local_file)})")
            return

        # 🔹 Check if `source_URL` is empty
        if not self.config.source_URL or self.config.source_URL.strip() == "":
            raise ValueError("❌ source_URL is missing or empty in config.yaml")

        try:
            logger.info(f"⬇️ Downloading file from {self.config.source_URL} ...")

            # 🔹 Debug print before download
            print(f"DEBUG: Attempting to download from {self.config.source_URL}")
            print(f"DEBUG: Saving to {local_file}")

            # 🔹 Open a request to the URL
            with request.urlopen(self.config.source_URL) as response:
                print(f"DEBUG: HTTP Response Code = {response.status}")
                if response.status != 200:
                    raise Exception(f"❌ HTTP Error: {response.status} while downloading {self.config.source_URL}")

                # 🔹 Save the file
                with open(local_file, "wb") as out_file:
                    shutil.copyfileobj(response, out_file)

            # 🔹 Verify file size after download
            print(f"DEBUG: Downloaded file size = {local_file.stat().st_size} bytes")
            if local_file.stat().st_size == 0:
                raise Exception(f"❌ Download failed: File is empty after downloading {self.config.source_URL}")

            logger.info(f"✅ Download complete: {local_file} ({get_size(local_file)})")

        except Exception as e:
            logger.error(f"❌ Download failed: {e}")
            raise Exception(f"Failed to download {self.config.source_URL}: {e}")
    def extract_zip_file(self):
        """Extract ZIP file using system tar command, with better error handling."""
        unzip_path = Path(self.config.unzip_dir)
        os.makedirs(unzip_path, exist_ok=True)
        local_zip = Path(self.config.local_data_file)

        if not local_zip.exists():
            raise FileNotFoundError(f"❌ File not found: {local_zip}")

        try:
            logger.info(f"📂 Extracting ZIP file using system `tar` command to {unzip_path} ...")

            # Run tar, but capture errors without stopping execution
            result = subprocess.run(
                ["tar", "-xf", str(local_zip), "-C", str(unzip_path)],
                capture_output=True,
                text=True
            )

            # 🔹 If tar returned an error but extracted files, ignore the error
            if result.returncode != 0:
                if any(Path(unzip_path).iterdir()):  # Check if files were extracted
                    logger.warning(f"⚠️ Extraction completed with warnings: {result.stderr}")
                else:
                    raise ValueError(f"❌ Extraction failed: {result.stderr}")

            logger.info(f"✅ Extraction complete using system tar command.")

        except Exception as e:
            raise ValueError(f"❌ Extraction failed: {e}")

In [37]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-02 19:50:18,911: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-02 19:50:18,914: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-02 19:50:18,916: INFO: common: created directory at: artifacts]
DEBUG: Loaded Config = {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/SAITEJAgitrez2/datasets/raw/refs/heads/main/samsumdata.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformat